In [9]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [10]:
import tkinter as tk
from tkinter import filedialog

def pick_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    return file_path

# location of the pdf file/files. 
# reader = pick_file()

In [11]:
import PyPDF2

# Open the PDF file in read-binary mode
with open(pick_file(), 'rb') as file:
    # Create a PyPDF2 PdfFileReader object
    reader = PyPDF2.PdfReader(file)

    # Read the text from each page and append it to the raw_text variable
    raw_text = ''
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        text = page.extract_text()
        if text:
            raw_text += text


In [15]:
raw_text[:100]

'  \n \n \n \n \n \n \n \n \nVertraulichkeitsvereinbarung  \n \nnachfolgend „Vereinbarung“  \n \n \nzwischen  \n \nAE'

In [12]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [17]:
len(texts)

9

In [18]:
texts[0]

'Vertraulichkeitsvereinbarung  \n \nnachfolgend „Vereinbarung“  \n \n \nzwischen  \n \nAEB SE \nSigmaringer Straße 109  \n7056 7 Stuttgart  \nDeutschland  \n \nund \n \nGiesecke+Devrient Group Services GmbH & Co. KG \nPrinzregentenstraße 159  \n81677 München  \nGermany  \n \nnachfolgend „ die Parteien “ \nVersion 0 8 08/2021 Vertraulichkeitsvereinbarung AEB - Vertragspartner  1  \nPräambel  \n \nIm Rahmen de r Anbah nung und Durchführung einer Geschäftsbeziehung zwischen den Parteien \nund der daraus resultierenden Kommunikation und Zusammenarbeit we rden den Parteien \nvertrauliche Informationen, Betriebs - oder Geschäftsgeheimnisse der jeweils anderen Partei \nbekannt. Deshalb treffen d ie Parteien für die gesamte n oben genannte n und weitere zukünftige \nAktivitäten  zwischen den Parteien folgende  \n \n \nGeheimhal tungsvereinbarung . \n \n1. Definitionen  \n \nA. Vertrauliche Informationen  \nAls vertraulich  im Sinne dieser Vereinbarung gilt eine  Information , die „Geschäftsgeh

In [13]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [14]:
docsearch = FAISS.from_texts(texts, embeddings)

In [22]:
docsearch

In [15]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [16]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [17]:
query = input("Question: ")
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Paperless Invoice is an additional service offered by UPS that allows for the electronic submission of a commercial invoice for a shipment. It eliminates the need for a physical document box and paper documents.'